# Clustering and Segmenting Neighborhoods in Toronto

<p>For the Toronto neighborhood data, a wikipedia page available in this link <a href="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M">here</a>.</p>

<p><b>1. Import Pandas</b><p>

In [1]:
import pandas as pd

<p>Install <b>html5lib</b> for reading Wikipedia Page to pandas</p>

In [2]:
!conda install html5lib -y

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /home/nbuser/anaconda3_420:

The following NEW packages will be INSTALLED:

    _libgcc_mutex: 0.1-main        
    html5lib:      1.0.1-py_0      
    readline:      7.0-ha6073c6_4  
    webencodings:  0.5.1-py35_1    

The following packages will be UPDATED:

    conda:         4.3.31-py35_0    --> 4.5.11-py35_0       
    conda-env:     2.6.0-h36134e3_1 --> 2.6.0-1             
    pycosat:       0.6.1-py35_1     --> 0.6.3-py35h6b6bb97_0

_libgcc_mutex- 100% |################################| Time: 0:00:00   9.03 kB/s
conda-env-2.6. 100% |################################| Time: 0:00:00   3.48 MB/s
readline-7.0-h 100% |################################| Time: 0:00:00   2.84 MB/s
pycosat-0.6.3- 100% |################################| Time: 0:00:00   2.67 MB/s
webencodings-0 100% |################################| Time: 0:00:00 871.54 kB/s
html5lib-1.0.1 100% |########

In [3]:
import html5lib

<p>'<strong>pd.read_html()</strong>' read all the tables in the wikipedia page!
</p>

In [4]:
#the URL containing the dataset
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

Assign the html content to a list named 'table'.

In [5]:
table=pd.read_html(url)
type(table)

list

<p>Type of table is list.
    We need only first element in the list list</p>

In [6]:
df=pd.DataFrame(table[0])

In [7]:
df.head()

,0,1,2
0,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


In [8]:
new_header=df.iloc[0]

df=df[1:]


In [9]:
df.columns=new_header

In [10]:
df[0:3]

,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods


<p>We need cells that have an assigned borough. Ignore cells with a borough that is <strong>Not assigned</strong>.</p>

In [11]:
#Check for the index value Not assigned in Borough column of df
index_borough=df[df['Borough'].isin(['Not assigned'])].index

#print indexes 
print(index_borough)

Int64Index([  1,   2,  10,  14,  21,  22,  31,  37,  38,  46,  47,  51,  52,
             53,  55,  56,  60,  61,  62,  74,  75,  76,  89,  90,  91, 105,
            106, 107, 121, 122, 137, 138, 149, 150, 156, 162, 163, 168, 176,
            182, 183, 189, 190, 191, 195, 196, 202, 203, 204, 205, 210, 211,
            224, 225, 238, 239, 242, 243, 248, 249, 254, 255, 259, 260, 261,
            262, 264, 265, 275, 276, 277, 278, 279, 280, 281, 282, 288],
           dtype='int64')


<p>Drop the cells with a borough <b>Not assigned</b></p>

In [12]:
df.drop(index_borough, inplace=True,axis=0)

<p>We can reset the row index in dataframe with reset_index() to make the index start from 0 and specify <b>drop=True</b> to not to keep the original index with the argument.</p>

In [13]:
df.reset_index(drop=True, inplace=True)
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


Inspect the Neighbourhood column for Not assigned value.

In [14]:
df.loc[df['Neighbourhood'].isin(['Not assigned'])]

,Postcode,Borough,Neighbourhood
6,M7A,Queen's Park,Not assigned


<p>Assign the Borough value to Neighbourhood, then the neighborhood will be the same as the borough. So for the 6th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.</p>

In [15]:
df.loc[df['Neighbourhood'].isin(['Not assigned']), 'Neighbourhood']=df['Borough']
print(df.iloc[6,:])

0
Postcode                  M7A
Borough          Queen's Park
Neighbourhood    Queen's Park
Name: 6, dtype: object


In [16]:
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


<p>Here, we can groupby on <b>'Postcode'</b> and <b>'Borough'</b> seeing as their relationship is the same, cast the <b>'Neighbourhood'</b> column to str and join with a comma:</p>

In [17]:
df_sort=df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(lambda x: ','.join(x.astype(str))).reset_index()

<p>Display the first 10 observations from the Wikipedia page grouped by Postcode</p>

In [18]:
df_sort.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [19]:
df_sort.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 3 columns):
Postcode         103 non-null object
Borough          103 non-null object
Neighbourhood    103 non-null object
dtypes: object(3)
memory usage: 2.5+ KB


In [20]:
df_sort.columns

Index(['Postcode', 'Borough', 'Neighbourhood'], dtype='object')

In [21]:
df_loc = pd.read_csv('Geospatial_Coordinates.csv')

In [22]:
df_loc.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [23]:
df_loc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 3 columns):
Postal Code    103 non-null object
Latitude       103 non-null float64
Longitude      103 non-null float64
dtypes: float64(2), object(1)
memory usage: 2.5+ KB


In [24]:
df_loc.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [69]:
df_loc.rename(columns={'Postal Code':'Postcode'}, inplace=True)
df_loc.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


###### Merging Dataframes 'df_sort' of Neighbourhood datasets and  'df_loc' of  locations dataset

In [72]:
df_merged=pd.merge(left=df_sort, right=df_loc, on='Postcode')

In [73]:
df_merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [74]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103 entries, 0 to 102
Data columns (total 5 columns):
Postcode         103 non-null object
Borough          103 non-null object
Neighbourhood    103 non-null object
Latitude         103 non-null float64
Longitude        103 non-null float64
dtypes: float64(2), object(3)
memory usage: 4.8+ KB
